# 贴现记录表分数

In [43]:
# -*- coding: utf-8 -*-
import pandas as pd

import xlrd,xlwt



df=pd.read_excel('实时报价.xls',encoding='GB18030')
#dataframe的分列处理
shijians=df['时间'].str.split(',',expand=True)
shijians.columns=['日期','时间2']#

shijians=shijians.reset_index(drop=True)
df=df.join(shijians)
df['时间2']=df['时间2'].str.replace('-',':')
df=df[['日期','时间2','票据类别','票据介质','期限品种','最低利率（%）','加权平均利率（%）','成交量(亿元)']]
#一，分析电银
df=df[(df['票据介质']=='电票') & (df['票据类别']=='银票')]

print(df.head())

df=df[['期限品种','日期','时间2','最低利率（%）']]


df=df.reset_index(drop=True)

shijian=df.loc[len(df)-1,'日期']

zixun_df=df[(df['日期']==str(shijian))]
zixun_df = zixun_df.fillna('')     
print(zixun_df)

if zixun_df.empty:
  yinhangjian_shun=[]
  yinhangjian_df=[]
else:
#zixun_df=zixun_df.drop_duplicates(['品种','时间'],keep='last')   #删除连接中的重复行


        yinhangjian_df = zixun_df['最低利率（%）'].groupby([zixun_df['期限品种'],zixun_df['时间2']]).sum().unstack('期限品种')

        yinhangjian_df=yinhangjian_df.reset_index(drop = False)

        #       yinhangjian_df=yinhangjian_df[[1,2,3,4,5]]  &&&&&&&&&&&&&&&&&&&



        #   print(yinhangjian_df)
        yinhangjian_df['时间3'] = pd.to_datetime(yinhangjian_df['时间2'])
        yinhangjian_df = yinhangjian_df.sort_values(by='时间3', ascending=False)
        del yinhangjian_df['时间3']
        yinhangjian_df=yinhangjian_df.reset_index(drop = True)

        yinhangjian_df=yinhangjian_df.set_index(['时间2'])
       # zuixiaozhi=yinhangjian_df.stack().min()
        zuixiaozhi='0'

      #  yinhangjian_df=yinhangjian_df.astype('float')
        #yinhangjian_df=yinhangjian_df.sort_index(ascending=False) 
        #

        shijiandian=yinhangjian_df.columns.values.tolist()  
        print(shijiandian)
        shijiandian1=[]
        shijiandian2=[]
        shijiandian3=[]
        for i in shijiandian:
         if 'M' in i:
             shijiandian1.append(i)
         elif 'Y' in i:
             shijiandian2.append(i)
         else:
             shijiandian3.append(i)



        shijiandian=shijiandian3+shijiandian1+shijiandian2

        shijiandian=shijiandian[:6]
        yinhangjian_df=yinhangjian_df[shijiandian]   #重命名列
        yinhangjian_df=yinhangjian_df.reset_index(drop = False)


        yinhangjian_shun=yinhangjian_df['时间2'].tolist()
        yinhangjian_df=yinhangjian_df.set_index('时间2').T.to_dict('list')
        print(yinhangjian_df)


           日期    时间2 票据类别 票据介质 期限品种  最低利率（%）  加权平均利率（%）  成交量(亿元)
4  2019-08-29  10:48   银票   电票   1M     2.75     2.9797   0.6530
5  2019-08-29  10:48   银票   电票   3M     2.55     2.9543   4.7869
6  2019-08-29  10:48   银票   电票   6M     2.40     3.2713   3.7973
7  2019-08-29  10:48   银票   电票   9M     2.65     3.1063   3.0462
8  2019-08-29  10:48   银票   电票   1Y     2.39     3.2219  28.9146
   期限品种          日期    时间2 最低利率（%）
45   1M  2019-08-30  09:06        
46   3M  2019-08-30  09:06        
47   6M  2019-08-30  09:06        
48   9M  2019-08-30  09:06        
49   1Y  2019-08-30  09:06        
50   1M  2019-08-30  09:22     3.2
51   3M  2019-08-30  09:22        
52   6M  2019-08-30  09:22        
53   9M  2019-08-30  09:22        
54   1Y  2019-08-30  09:22        
55   1M  2019-08-30  09:37     3.2
56   3M  2019-08-30  09:37        
57   6M  2019-08-30  09:37    2.68
58   9M  2019-08-30  09:37        
59   1Y  2019-08-30  09:37    2.46
60   1M  2019-08-30  09:52     3.2
61   3M  2019-0

In [40]:
#coding: utf_8_sig

import pandas as pd
import numpy as np
from numpy import NAN   #用来判断是否有NAN
from pandas import DataFrame
from pymongo import MongoClient
import time,datetime,json

client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')
import urllib.request





db23 = client.yinhangjian          
collection23 = db23.yinhangjian


cursor = collection23.find()
zixun_df= pd.DataFrame(list(cursor))
del zixun_df['_id']
zixun_df=zixun_df[['加权利率','品种','品种2','日期','时间','最新利率']]

shijian=zixun_df.loc[len(zixun_df)-1,'日期']

# shijian=shijian.strftime('%m-%d')
print(shijian)



zixun_df=zixun_df[(zixun_df['品种2']==1)]
yinhangjian2_df=zixun_df[(zixun_df['品种']=='DR001')|(zixun_df['品种']=='DR007')|(zixun_df['品种']=='DR014')]
yinhangjian2_df=yinhangjian2_df.drop_duplicates(['日期','品种'],keep='last')   #删除连接中的重复行

# yinhangjian2_df['日期'] = pd.to_datetime(yinhangjian2_df['日期']).astype('str')  #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
yinhangjian2_df = yinhangjian2_df['加权利率'].groupby([yinhangjian2_df['日期'],yinhangjian2_df['品种']]).sum().unstack('日期')

yinhangjian2_df=yinhangjian2_df.reset_index(drop = False)

datenames2=yinhangjian2_df.columns.values.tolist()[1:]
yinhangjian2_shun=yinhangjian2_df['品种'].tolist()
yinhangjian2_df=yinhangjian2_df.set_index('品种').T.to_dict('list')

print(yinhangjian2_df)
stop






zixun_df=zixun_df[(zixun_df['日期']==str(shijian))]

print(zixun_df)


if zixun_df.empty:
 yinhangjian_shun=[]
 yinhangjian_df=[]
else:
 zixun_df=zixun_df.drop_duplicates(['品种','时间'],keep='last')   #删除连接中的重复行


 yinhangjian_df = zixun_df['最新利率'].groupby([zixun_df['品种'],zixun_df['时间']]).sum().unstack('品种')

 yinhangjian_df=yinhangjian_df.reset_index(drop = False)

#       yinhangjian_df=yinhangjian_df[[1,2,3,4,5]]  &&&&&&&&&&&&&&&&&&&



#   print(yinhangjian_df)
 yinhangjian_df['时间2'] = pd.to_datetime(yinhangjian_df['时间'])
 yinhangjian_df = yinhangjian_df.sort_values(by='时间2', ascending=False)
 del yinhangjian_df['时间2']
 yinhangjian_df=yinhangjian_df.reset_index(drop = True)

 yinhangjian_df=yinhangjian_df.set_index(['时间'])
 zuixiaozhi=yinhangjian_df.stack().min()

 yinhangjian_df=yinhangjian_df.astype('float')
#yinhangjian_df=yinhangjian_df.sort_index(ascending=False) 
#

 shijiandian=yinhangjian_df.columns.values.tolist()  
 print(shijiandian)
 shijiandian1=[]
 shijiandian2=[]
 shijiandian3=[]
 for i in shijiandian:
     if 'M' in i:
         shijiandian1.append(i)
     elif 'Y' in i:
         shijiandian2.append(i)
     else:
         shijiandian3.append(i)



 shijiandian=shijiandian3+shijiandian1+shijiandian2

 shijiandian=shijiandian[:6]
 yinhangjian_df=yinhangjian_df[shijiandian]   #重命名列
 yinhangjian_df=yinhangjian_df.reset_index(drop = False)


 yinhangjian_shun=yinhangjian_df['时间'].tolist()
 yinhangjian_df=yinhangjian_df.set_index('时间').T.to_dict('list')

08-30
{'DR001': ['1.9969', '2.2375', '2.6165', '2.6096', '2.7302', '2.7957', '2.8283', '2.6576', '2.5541', '2.4759', '2.6272', '2.6255', '2.6234', '2.6158', '2.6568', '2.6329', '2.6027', '2.5258', '2.2559', '2.2158', '2.4948', '2.5967', '2.6345', '2.6819', '2.6911', '2.5897', '2.6613', '2.6663', '2.5500', '2.5807', '2.5500', '2.5864', '2.6414', '2.5500', '2.5500', '2.5692', '2.5764'], 'DR007': ['2.4284', '2.4925', '2.6633', '2.6738', '2.7707', '2.8056', '2.8324', '2.6918', '2.5960', '2.5301', '2.6350', '2.6389', '2.6579', '2.6712', '2.6944', '2.5878', '2.6355', '2.5885', '2.4312', '2.4285', '2.5391', '2.6273', '2.6614', '2.7379', '2.6965', '2.6500', '2.6731', '2.6745', '2.6500', '2.6424', '2.6500', '2.6427', '2.6826', '2.6500', '2.6500', '2.6500', '2.6691'], 'DR014': ['2.2771', '2.3487', '2.5981', '2.7000', '2.9057', '2.9922', '2.9766', '2.7964', '2.6707', '2.5753', '2.6421', '2.6959', '2.6676', '2.7286', '2.7224', '2.4390', '2.5588', '2.4443', '2.4123', '2.2982', '2.4062', '2.5007', '

NameError: name 'stop' is not defined